In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ReLU, AveragePooling2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.datasets import cifar10
from sklearn.metrics import f1_score

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
x_train.shape, x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [4]:
x_train = x_train.astype(np.float32) / 255.0
x_test = x_test.astype(np.float32) / 255.0

In [5]:
import numpy as np

class MySequential:
    def __init__(self):
        self.layers = []

    def add(self, layer):
        self.layers.append(layer)

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x

class Conv2DScratch:
    def __init__(self, weight, bias, stride=1, padding=0):
        self.weight = weight
        self.bias = bias
        self.stride = stride
        self.padding = padding

    def pad_input(self, x):
        if self.padding == 0:
            return x
        return np.pad(x, ((0, 0), (self.padding, self.padding), (self.padding, self.padding)), mode='constant')

    def forward(self, x):
        C_out, C_in, kH, kW = self.weight.shape
        x_padded = self.pad_input(x)
        _, H_in, W_in = x.shape
        H_out = (H_in + 2*self.padding - kH) // self.stride + 1
        W_out = (W_in + 2*self.padding - kW) // self.stride + 1

        out = np.zeros((C_out, H_out, W_out))

        for oc in range(C_out):
            for i in range(H_out):
                for j in range(W_out):
                    for ic in range(C_in):
                        h_start = i * self.stride
                        w_start = j * self.stride
                        patch = x_padded[ic, h_start:h_start+kH, w_start:w_start+kW]
                        out[oc, i, j] += np.sum(patch * self.weight[oc, ic])
                    out[oc, i, j] += self.bias[oc]
        return out

class ReLUScratch:
    def forward(self, x):
        return np.maximum(0, x)

class MaxPool2DScratch:
    def forward(self, x):
        # x: (C, H, W)
        C, H, W = x.shape
        out = np.zeros((C, H // 2, W // 2))
        for c in range(C):
            for i in range(0, H, 2):
                for j in range(0, W, 2):
                    out[c, i//2, j//2] = np.max(x[c, i:i+2, j:j+2])
        return out

class AveragePool2DScratch:
    def forward(self, x):
        # x: (C, H, W)
        C, H, W = x.shape
        out = np.zeros((C, H // 2, W // 2))
        for c in range(C):
            for i in range(0, H, 2):
                for j in range(0, W, 2):
                    out[c, i//2, j//2] = np.mean(x[c, i:i+2, j:j+2])
        return out

class FlattenScratch:
    def forward(self, x):
        return x.flatten()

class DenseScratch:
    def __init__(self, weight, bias):
        self.weight = weight
        self.bias = bias

    def forward(self, x):
        return self.weight @ x + self.bias

class SoftmaxScratch:
    def forward(self, x):
        x_shifted = x - np.max(x)
        exp_x = np.exp(x_shifted)
        return exp_x / np.sum(exp_x)



## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 2, MaxPooling

### Library Inititiation

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_6 (ReLU)                  │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_7 (ReLU)                  │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,858 (30.70 KB)

 Trainable params: 7,858 (30.70 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 42s 58ms/step - accuracy: 0.2653 - loss: 2.0446 - val_accuracy: 0.3852 - val_loss: 1.7710
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.4091 - loss: 1.6906 - val_accuracy: 0.4410 - val_loss: 1.6213
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 39s 48ms/step - accuracy: 0.4609 - loss: 1.5492 - val_accuracy: 0.4590 - val_loss: 1.5406
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 47ms/step - accuracy: 0.4790 - loss: 1.4986 - val_accuracy: 0.4706 - val_loss: 1.5072
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 47ms/step - accuracy: 0.4836 - loss: 1.4726 - val_accuracy: 0.4750 - val_loss: 1.4864


### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.46777719439512644


### Scratch Predictions

In [ ]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [ ]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [ ]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [ ]:
data = np.load("trained_cnn_data.npz")

In [ ]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [ ]:
y_pred = []

for i in range(len(x_test)):
    img_np = x_test[i]
    img_scratch = img_np.transpose(2, 0, 1)
    out = myModel.forward(img_scratch)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 2222 samples
Label 1: 3667 samples
Label 2: 260 samples
Label 3: 170 samples
Label 4: 2 samples
Label 5: 1 samples
Label 6: 16 samples
Label 7: 2 samples
Label 8: 187 samples
Label 9: 3473 samples


In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.08269827603505234


## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 2, AveragePooling

### Library Inititalization

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    AveragePooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_10 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_11 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (AveragePooling2D)         │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_5 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,858 (30.70 KB)

 Trainable params: 7,858 (30.70 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 47ms/step - accuracy: 0.2616 - loss: 2.0159 - val_accuracy: 0.3856 - val_loss: 1.7373
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 39s 45ms/step - accuracy: 0.4036 - loss: 1.6918 - val_accuracy: 0.4118 - val_loss: 1.6537
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.4220 - loss: 1.6302 - val_accuracy: 0.4370 - val_loss: 1.6018
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 32s 45ms/step - accuracy: 0.4389 - loss: 1.6011 - val_accuracy: 0.4422 - val_loss: 1.5733
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 40s 43ms/step - accuracy: 0.4559 - loss: 1.5604 - val_accuracy: 0.4564 - val_loss: 1.5469


### Library Prediction

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.47572486486052645


### Scratch Prediction

In [ ]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [ ]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [ ]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [ ]:
data = np.load("trained_cnn_data.npz")

In [ ]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(AveragePool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [ ]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 122 samples
Label 1: 2 samples
Label 2: 9087 samples
Label 3: 15 samples
Label 4: 758 samples
Label 5: 0 samples
Label 6: 4 samples
Label 7: 0 samples
Label 8: 12 samples
Label 9: 0 samples


In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.042656360762554545


## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 3, MaxPooling

### Library Initiation

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv3'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_12 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_13 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_14 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,942 (31.02 KB)

 Trainable params: 7,942 (31.02 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 50s 69ms/step - accuracy: 0.2145 - loss: 2.1138 - val_accuracy: 0.3676 - val_loss: 1.7729
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 48s 69ms/step - accuracy: 0.3957 - loss: 1.7164 - val_accuracy: 0.4232 - val_loss: 1.6329
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 48s 68ms/step - accuracy: 0.4300 - loss: 1.6149 - val_accuracy: 0.4036 - val_loss: 1.6885
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 47s 67ms/step - accuracy: 0.4537 - loss: 1.5538 - val_accuracy: 0.4570 - val_loss: 1.5168
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 48s 68ms/step - accuracy: 0.4697 - loss: 1.5151 - val_accuracy: 0.4680 - val_loss: 1.4934


### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.48579990542399243


### Scratch Predictions

In [ ]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
conv3_w, conv3_b = model.get_layer('conv3').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [ ]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)
conv3_w_nchw = conv3_w.transpose(3, 2, 0, 1)

In [ ]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         conv3_w=conv3_w_nchw, conv3_b=conv3_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [ ]:
data = np.load("trained_cnn_data.npz")

In [ ]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv3_w"], data["conv3_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [ ]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 6494 samples
Label 1: 806 samples
Label 2: 1259 samples
Label 3: 1018 samples
Label 4: 226 samples
Label 5: 2 samples
Label 6: 5 samples
Label 7: 63 samples
Label 8: 7 samples
Label 9: 120 samples


In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.07775405312850872


## Filters = 3, padding = 1, kernel-size = 3x3, n_conv_layer = 4, MaxPooling

### Library Initiation

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv2'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv3'),
    ReLU(),
    Conv2D(3, (3, 3), padding='same', name='conv4'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_15 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_16 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_17 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv4 (Conv2D)                  │ (None, 32, 32, 3)      │            84 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_18 (ReLU)                 │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_7 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,026 (31.35 KB)

 Trainable params: 8,026 (31.35 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 67s 91ms/step - accuracy: 0.2647 - loss: 2.0106 - val_accuracy: 0.4240 - val_loss: 1.6307
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 80s 88ms/step - accuracy: 0.4333 - loss: 1.6117 - val_accuracy: 0.4508 - val_loss: 1.5475
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 81s 87ms/step - accuracy: 0.4612 - loss: 1.5375 - val_accuracy: 0.4744 - val_loss: 1.5002
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 88ms/step - accuracy: 0.4721 - loss: 1.4968 - val_accuracy: 0.4760 - val_loss: 1.4617
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 62s 88ms/step - accuracy: 0.4908 - loss: 1.4482 - val_accuracy: 0.4824 - val_loss: 1.4417


### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.47540873316105065


### Scratch Predictions

In [ ]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
conv3_w, conv3_b = model.get_layer('conv3').get_weights()
conv4_w, conv4_b = model.get_layer('conv4').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [ ]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)
conv3_w_nchw = conv3_w.transpose(3, 2, 0, 1)
conv4_w_nchw = conv4_w.transpose(3, 2, 0, 1)

In [ ]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         conv3_w=conv3_w_nchw, conv3_b=conv3_b,
         conv4_w=conv4_w_nchw, conv4_b=conv4_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [ ]:
data = np.load("trained_cnn_data.npz")

In [ ]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv3_w"], data["conv3_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv4_w"], data["conv4_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [ ]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 20 samples
Label 1: 9 samples
Label 2: 6804 samples
Label 3: 1248 samples
Label 4: 197 samples
Label 5: 62 samples
Label 6: 0 samples
Label 7: 1660 samples
Label 8: 0 samples
Label 9: 0 samples


In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.044742280611098556


## Filters = 2, padding = 1, kernel-size = 3x3, n_conv_layer = 2, MaxPooling

### Library Initialization

In [ ]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(2, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(2, (3, 3), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 2)      │            56 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_19 (ReLU)                 │ (None, 32, 32, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 2)      │            38 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_20 (ReLU)                 │ (None, 32, 32, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 2)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_8 (Flatten)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,224 (20.41 KB)

 Trainable params: 5,224 (20.41 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 32s 44ms/step - accuracy: 0.2438 - loss: 2.0605 - val_accuracy: 0.4194 - val_loss: 1.6646
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 31s 44ms/step - accuracy: 0.4165 - loss: 1.6492 - val_accuracy: 0.4470 - val_loss: 1.5929
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.4398 - loss: 1.5821 - val_accuracy: 0.4398 - val_loss: 1.5706
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 44ms/step - accuracy: 0.4519 - loss: 1.5450 - val_accuracy: 0.4486 - val_loss: 1.5520
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 43ms/step - accuracy: 0.4633 - loss: 1.5169 - val_accuracy: 0.4538 - val_loss: 1.5350


### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.3895190368551136


### Scratch Predictions

In [ ]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [ ]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [ ]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [ ]:
data = np.load("trained_cnn_data.npz")

In [ ]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [ ]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [ ]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 118 samples
Label 1: 4879 samples
Label 2: 63 samples
Label 3: 226 samples
Label 4: 1737 samples
Label 5: 705 samples
Label 6: 1104 samples
Label 7: 565 samples
Label 8: 39 samples
Label 9: 564 samples


In [ ]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.08370834384183609


## Filters = 4, padding = 1, kernel-size = 3x3, n_conv_layer = 2, MaxPooling

### Library Initialization

In [142]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(4, (3, 3), padding='same', name='conv1'),
    ReLU(),
    Conv2D(4, (3, 3), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 4)      │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_23 (ReLU)                 │ (None, 32, 32, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 4)      │           148 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_24 (ReLU)                 │ (None, 32, 32, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 4)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │        10,250 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 10,510 (41.05 KB)

 Trainable params: 10,510 (41.05 KB)

 Non-trainable params: 0 (0.00 B)

In [143]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [144]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 48s 55ms/step - accuracy: 0.2939 - loss: 1.9527 - val_accuracy: 0.4398 - val_loss: 1.6059
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 34s 49ms/step - accuracy: 0.4477 - loss: 1.5853 - val_accuracy: 0.4760 - val_loss: 1.4938
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.4895 - loss: 1.4510 - val_accuracy: 0.5028 - val_loss: 1.4223
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 42s 50ms/step - accuracy: 0.5101 - loss: 1.3944 - val_accuracy: 0.5190 - val_loss: 1.3887
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.5279 - loss: 1.3457 - val_accuracy: 0.5294 - val_loss: 1.3639


### Library Predictions

In [145]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [146]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.5132189270901975


### Scratch Predictions

In [147]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [148]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [149]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [150]:
data = np.load("trained_cnn_data.npz")

In [151]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=1))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [152]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [153]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 767 samples
Label 1: 1691 samples
Label 2: 2533 samples
Label 3: 1052 samples
Label 4: 97 samples
Label 5: 214 samples
Label 6: 4 samples
Label 7: 598 samples
Label 8: 1 samples
Label 9: 3043 samples


In [154]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.06607091350599484


## Filters = 3, padding = 2, kernel-size = 5x5, n_conv_layer = 2, MaxPooling

### Library Initialization

In [6]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (5, 5), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (5, 5), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │           228 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,146 (31.82 KB)

 Trainable params: 8,146 (31.82 KB)

 Non-trainable params: 0 (0.00 B)

In [7]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [8]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 61s 85ms/step - accuracy: 0.2682 - loss: 2.0144 - val_accuracy: 0.3916 - val_loss: 1.7292
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 83s 87ms/step - accuracy: 0.4274 - loss: 1.6613 - val_accuracy: 0.4250 - val_loss: 1.6418
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 82s 87ms/step - accuracy: 0.4556 - loss: 1.5694 - val_accuracy: 0.4576 - val_loss: 1.5527
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 59s 84ms/step - accuracy: 0.4713 - loss: 1.5328 - val_accuracy: 0.4688 - val_loss: 1.5185
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 87s 91ms/step - accuracy: 0.4786 - loss: 1.5052 - val_accuracy: 0.4634 - val_loss: 1.5405


### Library Predictions

In [ ]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.47825036080672445


### Scratch Predictions

In [9]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [10]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [11]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [12]:
data = np.load("trained_cnn_data.npz")

In [13]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=2))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=2))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [14]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [15]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 0 samples
Label 1: 2 samples
Label 2: 56 samples
Label 3: 6411 samples
Label 4: 731 samples
Label 5: 49 samples
Label 6: 1562 samples
Label 7: 1189 samples
Label 8: 0 samples
Label 9: 0 samples


In [16]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.047114289145776186


## Filters = 3, padding = 1, kernel-size = 7x7, n_conv_layer = 2, MaxPooling

### Library Initialization

In [17]:
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(3, (7, 7), padding='same', name='conv1'),
    ReLU(),
    Conv2D(3, (7, 7), padding='same', name='conv2'),
    ReLU(),
    MaxPooling2D(pool_size=2, strides=2, name='pool'),
    Flatten(),
    Dense(10, activation='softmax', name='fc')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 32, 32, 3)      │           444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 32, 32, 3)      │           444 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_3 (ReLU)                  │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ pool (MaxPooling2D)             │ (None, 16, 16, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 768)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc (Dense)                      │ (None, 10)             │         7,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,578 (33.51 KB)

 Trainable params: 8,578 (33.51 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [168]:
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 104s 146ms/step - accuracy: 0.2844 - loss: 1.9525 - val_accuracy: 0.3896 - val_loss: 1.6749
Epoch 2/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 101s 144ms/step - accuracy: 0.4324 - loss: 1.5910 - val_accuracy: 0.4458 - val_loss: 1.5522
Epoch 3/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 103s 146ms/step - accuracy: 0.4620 - loss: 1.5192 - val_accuracy: 0.4742 - val_loss: 1.4934
Epoch 4/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 101s 144ms/step - accuracy: 0.4828 - loss: 1.4646 - val_accuracy: 0.4894 - val_loss: 1.4541
Epoch 5/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 103s 146ms/step - accuracy: 0.4928 - loss: 1.4378 - val_accuracy: 0.5038 - val_loss: 1.4051


### Library Predictions

In [19]:
predictions = model.predict(x_test)
o_y_pred = np.argmax(predictions, axis=1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step


In [ ]:
o_macro_f1 = f1_score(y_test.flatten(), o_y_pred, average='macro')
print("Macro F1-score:", o_macro_f1)

Macro F1-score: 0.433556296723384


### Scratch Predictions

In [20]:
conv1_w, conv1_b = model.get_layer('conv1').get_weights()
conv2_w, conv2_b = model.get_layer('conv2').get_weights()
fc_w, fc_b = model.get_layer('fc').get_weights()

In [21]:
# Convert conv weights to NCHW
conv1_w_nchw = conv1_w.transpose(3, 2, 0, 1)
conv2_w_nchw = conv2_w.transpose(3, 2, 0, 1)

In [22]:
np.savez("trained_cnn_data.npz",
         conv1_w=conv1_w_nchw, conv1_b=conv1_b,
         conv2_w=conv2_w_nchw, conv2_b=conv2_b,
         fc_w=fc_w.T, fc_b=fc_b)  # fc_w: (10, flattened_size)

In [23]:
data = np.load("trained_cnn_data.npz")

In [24]:
myModel = MySequential()

myModel.add(Conv2DScratch(data["conv1_w"], data["conv1_b"], padding=3))
myModel.add(ReLUScratch())
myModel.add(Conv2DScratch(data["conv2_w"], data["conv2_b"], padding=3))
myModel.add(ReLUScratch())
myModel.add(MaxPool2DScratch())
myModel.add(FlattenScratch())
myModel.add(DenseScratch(data["fc_w"], data["fc_b"]))
myModel.add(SoftmaxScratch())

In [25]:
y_pred = []

for i in range(len(x_test)):
    img = x_test[i].transpose(2, 0, 1)
    out = myModel.forward(img)
    pred = np.argmax(out)
    y_pred.append(pred)

y_pred = np.array(y_pred)

In [26]:
label_counts = np.bincount(y_pred, minlength=10)

for label, count in enumerate(label_counts):
    print(f"Label {label}: {count} samples")

Label 0: 318 samples
Label 1: 81 samples
Label 2: 3337 samples
Label 3: 406 samples
Label 4: 1276 samples
Label 5: 624 samples
Label 6: 2812 samples
Label 7: 207 samples
Label 8: 206 samples
Label 9: 733 samples


In [27]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
print("Macro F1-score:", macro_f1)

Macro F1-score: 0.07573150294509395
